In [76]:
import csv
import pandas as pd
import numpy as np
import os

In [77]:
cwd = os.getcwd() # TO GET CURRENT WORKING DIRECTORY, SO THAT WE CAN NAVIGATE INTO INNER FOLDERS

In [89]:
states_df = {}                  # DICTIONARY WITH STATE NAME AS KEY AND DATAFRAME AS VALUE
mother_tongue_states_df = {}
path = cwd +'\\C-17\\'          # INNER FOLDER NAME WHERE THE DATA LIES
fileList = os.listdir(path)
for file in fileList:           # ITERATING ALL FILES IN OUR DIRECTORY C-17
    file_name = path + file     # CONSTRUCTING ENTIRE FILE PATH TO READ FILE
    df = pd.read_excel(file_name) # READING EXCEL FILE INTO OUR DATAFRAME
    df = df.iloc[5:]            # REMOVING UNWANTED ROWS 
    state_name = df.iloc[0][1]  # EXTRACTING STATE NAME
    df.drop(df.columns[[0,1,2,5,6,7,10,11,12,15,16]], axis=1, inplace=True) # REMOVING ALL OTHER UNUSEFUL COLUMNS
    # WE ARE NOW LEFT WITH SIX COLUMNS, DIVIDING TWO COLUMNS IN EACH DATAFRAME
    df1 = df.iloc[:, 0:2]        # TOTAL LANGUAGE SPEAKERS AND CORRESPONDING LANGUAGE
    mother_tongue_df = df1.values
    df2 = df.iloc[:, 2:4]        # SUBCIDIARY FIRST LANGUAGE SPEAKERS AND CORRESPONDING LANGUAGE
    df3 = df.iloc[:, 4:6]        # SUBCIDIARY SECOND LANGUAGE SPEAKERS AND CORRESPONDING LANGUAGE
    df = pd.DataFrame( np.concatenate( (df1.values, df2.values,df3.values), axis=0 ) ) # CONCATENATING ALL DATAFRAMES
    df = df.dropna()             # DROPPING ROWS WITH MISSING VALUES
    df.columns = [ 'AllSpeakersLang', 'AllSpeakersLangPersons' ]     # RENAMING COLUMNS WITH CONVENINIENT NAMES
    mother_tongue_df = pd.DataFrame(mother_tongue_df, columns = [ 'AllSpeakersLang', 'AllSpeakersLangPersons' ] ) 
    mother_tongue_df = mother_tongue_df.dropna()
    df = df.groupby(['AllSpeakersLang'],as_index=False).agg({'AllSpeakersLangPersons':'sum'}) # AGGREAGATING ALL SPEAKERS OF LANGUAGES BASED ON LANGUAGE
    mother_tongue_df = mother_tongue_df.groupby(['AllSpeakersLang'],as_index=False).agg({'AllSpeakersLangPersons':'sum'})
    states_df[state_name] = df
    mother_tongue_states_df[state_name] = mother_tongue_df

In [79]:
states_df.keys()  # ALL STATE NAMES

dict_keys(['JAMMU & KASHMIR', 'HIMACHAL PRADESH', 'PUNJAB', 'CHANDIGARH', 'UTTARAKHAND', 'HARYANA', 'NCT OF DELHI', 'RAJASTHAN', 'UTTAR PRADESH', 'BIHAR', 'SIKKIM', 'ARUNACHAL PRADESH', 'NAGALAND', 'MANIPUR', 'MIZORAM', 'TRIPURA', 'MEGHALAYA', 'ASSAM', 'WEST BENGAL', 'JHARKHAND', 'ODISHA', 'CHHATTISGARH', 'MADHYA PRADESH', 'GUJARAT', 'DAMAN & DIU', 'DADRA & NAGAR HAVELI', 'MAHARASHTRA', 'ANDHRA PRADESH', 'KARNATAKA', 'GOA', 'LAKSHADWEEP', 'KERALA', 'TAMIL NADU', 'PUDUCHERRY', 'ANDAMAN & NICOBAR ISLANDS'])

In [80]:
region_df = {}        # DICTIONARY WITH REGION NAME AS KEY AND DATAFRAME AS VALUE

# AGGREGATING ALL STATES INTO RESPECTIVE REGION
region_df['North'] = pd.DataFrame( np.concatenate( (states_df['JAMMU & KASHMIR'].values, states_df['PUNJAB'].values, 
                                                    states_df['HIMACHAL PRADESH'].values, states_df['HARYANA'].values,
                                                   states_df['UTTARAKHAND'].values, states_df['NCT OF DELHI'].values,
                                                   states_df['CHANDIGARH'].values), axis=0 ) )
region_df['West'] = pd.DataFrame( np.concatenate( (states_df['RAJASTHAN'].values, states_df['GUJARAT'].values, 
                                                    states_df['MAHARASHTRA'].values, states_df['GOA'].values,
                                                   states_df['DADRA & NAGAR HAVELI'].values, states_df['DAMAN & DIU'].values),axis=0 ) )
region_df['Central'] = pd.DataFrame( np.concatenate( (states_df['MADHYA PRADESH'].values, states_df['UTTAR PRADESH'].values, 
                                                    states_df['CHHATTISGARH'].values), axis=0 ) )
region_df['East'] = pd.DataFrame( np.concatenate( (states_df['BIHAR'].values, states_df['WEST BENGAL'].values, 
                                                    states_df['ODISHA'].values, states_df['JHARKHAND'].values),
                                                    axis=0 ) )
region_df['South'] = pd.DataFrame( np.concatenate( (states_df['KARNATAKA'].values, states_df['ANDHRA PRADESH'].values, 
                                                    states_df['TAMIL NADU'].values, states_df['KERALA'].values,
                                                    states_df['LAKSHADWEEP'].values, states_df['PUDUCHERRY'].values),
                                                    axis=0 ) )
region_df['North-East'] = pd.DataFrame( np.concatenate( (states_df['ASSAM'].values, states_df['SIKKIM'].values, 
                                                         states_df['MEGHALAYA'].values,
                                                    states_df['TRIPURA'].values, states_df['ARUNACHAL PRADESH'].values,
                                                   states_df['MANIPUR'].values, states_df['NAGALAND'].values,
                                                   states_df['MIZORAM'].values, states_df['ANDAMAN & NICOBAR ISLANDS'].values),
                                                       axis=0 ) )                                  

In [81]:
final_df = pd.DataFrame(columns = ['region','language-1','language-2','language-3'])  # FINAL OUTPUR DATAFRAME
for i in region_df.keys():   # ITERATING THROUGH ALL REGIONS
    region_df[i].columns = [ 'AllSpeakersLang', 'AllSpeakersLangPersons' ]  # NAMING COLUMNS WITH OUR CONVENTION
    region_df[i] = region_df[i].groupby(['AllSpeakersLang'],as_index=False).agg({'AllSpeakersLangPersons':'sum'}) # AGGREGATING ALL SPEAKERS OF LANGUAGES BASED ON LANGUAGE
    region_df[i] = region_df[i].sort_values(by=['AllSpeakersLangPersons'], ascending=False)  # SORTING IN DESCENDING ORDER TO GET MOST SPOKEN LANGUAGES
    # APPENDING THE REGION VALUES TO THE FINAL DATAFRAME
    final_df = final_df.append(pd.DataFrame({"region": i,"language-1":region_df[i]['AllSpeakersLang'].values[0],"language-2":region_df[i]['AllSpeakersLang'].values[1],'language-3':region_df[i]['AllSpeakersLang'].values[2]}, index=[0]))
final_df = final_df.sort_values(by=['region'], ascending=True)   

In [82]:
final_df.to_csv(r'region-india-b.csv', index = None, header=True) # WRITING OUR DATAFRAME INTO OUTPUT CSV FILE

In [83]:
states_df=mother_tongue_states_df
region_df = {}        # DICTIONARY WITH REGION NAME AS KEY AND DATAFRAME AS VALUE

# AGGREGATING ALL STATES INTO RESPECTIVE REGION
region_df['North'] = pd.DataFrame( np.concatenate( (states_df['JAMMU & KASHMIR'].values, states_df['PUNJAB'].values, 
                                                    states_df['HIMACHAL PRADESH'].values, states_df['HARYANA'].values,
                                                   states_df['UTTARAKHAND'].values, states_df['NCT OF DELHI'].values,
                                                   states_df['CHANDIGARH'].values), axis=0 ) )
region_df['West'] = pd.DataFrame( np.concatenate( (states_df['RAJASTHAN'].values, states_df['GUJARAT'].values, 
                                                    states_df['MAHARASHTRA'].values, states_df['GOA'].values,
                                                   states_df['DADRA & NAGAR HAVELI'].values, states_df['DAMAN & DIU'].values),axis=0 ) )
region_df['Central'] = pd.DataFrame( np.concatenate( (states_df['MADHYA PRADESH'].values, states_df['UTTAR PRADESH'].values, 
                                                    states_df['CHHATTISGARH'].values), axis=0 ) )
region_df['East'] = pd.DataFrame( np.concatenate( (states_df['BIHAR'].values, states_df['WEST BENGAL'].values, 
                                                    states_df['ODISHA'].values, states_df['JHARKHAND'].values),
                                                    axis=0 ) )
region_df['South'] = pd.DataFrame( np.concatenate( (states_df['KARNATAKA'].values, states_df['ANDHRA PRADESH'].values, 
                                                    states_df['TAMIL NADU'].values, states_df['KERALA'].values,
                                                    states_df['LAKSHADWEEP'].values, states_df['PUDUCHERRY'].values),
                                                    axis=0 ) )
region_df['North-East'] = pd.DataFrame( np.concatenate( (states_df['ASSAM'].values, states_df['SIKKIM'].values, 
                                                         states_df['MEGHALAYA'].values,
                                                    states_df['TRIPURA'].values, states_df['ARUNACHAL PRADESH'].values,
                                                   states_df['MANIPUR'].values, states_df['NAGALAND'].values,
                                                   states_df['MIZORAM'].values, states_df['ANDAMAN & NICOBAR ISLANDS'].values),
                                                       axis=0 ) )                                  

In [84]:
final_mother_tongue_df = pd.DataFrame(columns = ['region','language-1','language-2','language-3'])  # FINAL OUTPUR DATAFRAME
for i in region_df.keys():   # ITERATING THROUGH ALL REGIONS
    region_df[i].columns = [ 'AllSpeakersLang', 'AllSpeakersLangPersons' ]  # NAMING COLUMNS WITH OUR CONVENTION
    region_df[i] = region_df[i].groupby(['AllSpeakersLang'],as_index=False).agg({'AllSpeakersLangPersons':'sum'}) # AGGREGATING ALL SPEAKERS OF LANGUAGES BASED ON LANGUAGE
    region_df[i] = region_df[i].sort_values(by=['AllSpeakersLangPersons'], ascending=False)  # SORTING IN DESCENDING ORDER TO GET MOST SPOKEN LANGUAGES
    # APPENDING THE REGION VALUES TO THE FINAL DATAFRAME
    final_mother_tongue_df = final_mother_tongue_df.append(pd.DataFrame({"region": i,"language-1":region_df[i]['AllSpeakersLang'].values[0],"language-2":region_df[i]['AllSpeakersLang'].values[1],'language-3':region_df[i]['AllSpeakersLang'].values[2]}, index=[0]))
final_mother_tongue_df = final_mother_tongue_df.sort_values(by=['region'], ascending=True) 

In [85]:
final_mother_tongue_df.to_csv(r'region-india-a.csv', index = None, header=True) # WRITING OUR DATAFRAME INTO OUTPUT CSV FILE